In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE_DIR = Path(".")

# Girdiler
SERVICE_PLAN_FILE = BASE_DIR / "service_plan_by_date.csv"
CONTAINER_FILE    = BASE_DIR / "container_counts.csv"
POP_FILE          = BASE_DIR / "mahalle_nufus.csv"

# Opsiyonel (varsa kullanır): Faz 1 çıktın
BASELINE_NEIGH_FILE = BASE_DIR / "baseline_daily_neighborhood_metrics.csv"

# Çıktılar (aynı klasöre)
OUT_DEMAND = BASE_DIR / "demand_by_date_neighborhood.csv"
OUT_COMP   = BASE_DIR / "demand_components_neighborhood.csv"
OUT_SUM    = BASE_DIR / "demand_summary_by_date.csv"

# -------------------------
# Yardımcı fonksiyonlar
# -------------------------
def norm_mahalle(x: str) -> str:
    """Mahalle isimlerini join için agresif normalize et."""
    if pd.isna(x):
        return ""
    s = str(x).strip().upper()

    # Türkçe karakterleri normalize et (eşleşme artırır)
    tr_map = str.maketrans({"Ç":"C","Ş":"S","Ğ":"G","İ":"I","I":"I","Ö":"O","Ü":"U","ı":"I","ç":"C","ş":"S","ğ":"G","ö":"O","ü":"U"})
    s = s.translate(tr_map)

    # yaygın ek/format farklarını temizle
    # ör: "GÖRÜKLE MAHALLESİ" vs "GÖRÜKLE"
    for token in [" MAHALLESI", " MAHALLESİ", " MAHALLE", " MH.", " MH", " MAH.", " MAH"]:
        s = s.replace(token, "")

    # çoklu boşlukları temizle
    s = " ".join(s.split())
    return s

def zscore(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean(skipna=True)
    sd = s.std(skipna=True)
    if pd.isna(sd) or sd == 0:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / sd

def minmax01(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    mn, mx = s.min(skipna=True), s.max(skipna=True)
    if pd.isna(mx) or mx == mn:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mn) / (mx - mn)

# -------------------------
# 1) Service plan oku
# -------------------------
sp = pd.read_csv(SERVICE_PLAN_FILE)
sp["Mahalle_norm"] = sp["Mahalle"].apply(norm_mahalle)

# -------------------------
# 2) Konteyner verisini oku (sep=';')
# -------------------------
cc = pd.read_csv(CONTAINER_FILE, sep=";", engine="python")
cc.columns = [c.strip() for c in cc.columns]

if "MAHALLE" not in cc.columns:
    raise ValueError(f"container_counts.csv kolonları beklenmedik: {cc.columns.tolist()}")

cc["Mahalle_norm"] = cc["MAHALLE"].apply(norm_mahalle)

# sayısal kolonları parse et
num_cols = [c for c in cc.columns if c not in ["SIRA NO", "MAHALLE", "Mahalle_norm"]]
for c in num_cols:
    cc[c] = pd.to_numeric(cc[c], errors="coerce")

# Konteyner bileşen kolonları (dosyanın tipik yapısına göre)
component_candidates = [
    "YERALTI KONTEYNER",
    "770 LT KONTEYNER",
    "400 LT KONTEYNER",
    "PLASTİK"
]
component_cols = [c for c in component_candidates if c in cc.columns]

# TOPLAM varsa al, ama 0/NaN çoksa component toplamına fallback yap
if "TOPLAM" in cc.columns:
    cc["container_total_toplam"] = cc["TOPLAM"]
else:
    cc["container_total_toplam"] = np.nan

if component_cols:
    cc["container_total_components"] = cc[component_cols].sum(axis=1, skipna=True)
else:
    # komponent kolonları yoksa, tüm sayısal kolonları topla (TOPLAM hariç)
    fallback_cols = [c for c in num_cols if c != "TOPLAM"]
    cc["container_total_components"] = cc[fallback_cols].sum(axis=1, skipna=True)

# Final container_total seçimi:
# - TOPLAM dolu ve anlamlıysa onu kullan
# - değilse component toplamına dön
cc["container_total"] = cc["container_total_toplam"]
cc.loc[cc["container_total"].isna() | (cc["container_total"] <= 0), "container_total"] = cc["container_total_components"]

# mahalle bazında birleştir
cc_neigh = cc.groupby("Mahalle_norm", as_index=False)["container_total"].sum()

# -------------------------
# 3) Nüfus verisini oku (sep=';')
# -------------------------
mn = pd.read_csv(POP_FILE, sep=";", engine="python")
mn.columns = [c.strip().lower() for c in mn.columns]

if "mahalle" not in mn.columns or "nufus" not in mn.columns:
    raise ValueError(f"mahalle_nufus.csv kolonları beklenmedik: {mn.columns.tolist()}")

mn["Mahalle_norm"] = mn["mahalle"].apply(norm_mahalle)

mn["population"] = (
    mn["nufus"].astype(str)
      .str.replace(".", "", regex=False)   # binlik noktayı kaldır
      .str.replace(",", ".", regex=False)
)
mn["population"] = pd.to_numeric(mn["population"], errors="coerce")
mn_neigh = mn.groupby("Mahalle_norm", as_index=False)["population"].max()

# -------------------------
# 4) (Opsiyonel) Faz 1 baseline yoğunluğu
# -------------------------
baseline_neigh = None
if BASELINE_NEIGH_FILE.exists():
    b = pd.read_csv(BASELINE_NEIGH_FILE)
    if "Mahalle" in b.columns:
        b["Mahalle_norm"] = b["Mahalle"].apply(norm_mahalle)

        cand = None
        for c in ["total_time_in_neigh_sec", "total_dist_in_neigh_km", "vehicles_served"]:
            if c in b.columns:
                cand = c
                break

        if cand is not None:
            baseline_neigh = (
                b.groupby("Mahalle_norm", as_index=False)[cand]
                 .mean()
                 .rename(columns={cand: "baseline_activity"})
            )

# -------------------------
# 5) Birleştir + eşleşme kalitesini raporla
# -------------------------
neigh = pd.DataFrame({"Mahalle_norm": sp["Mahalle_norm"].unique()})

neigh = neigh.merge(cc_neigh, on="Mahalle_norm", how="left")
neigh = neigh.merge(mn_neigh, on="Mahalle_norm", how="left")
if baseline_neigh is not None:
    neigh = neigh.merge(baseline_neigh, on="Mahalle_norm", how="left")
else:
    neigh["baseline_activity"] = np.nan

# Eşleşme teşhisi (çok önemli)
match_container_rate = 1 - neigh["container_total"].isna().mean()
match_pop_rate = 1 - neigh["population"].isna().mean()
print(f"[FAZ3.1] container eşleşme oranı: {match_container_rate:.2%}")
print(f"[FAZ3.1] nüfus eşleşme oranı:     {match_pop_rate:.2%}")

# Eksikleri 0 ile doldur
neigh["container_total"] = neigh["container_total"].fillna(0.0)
neigh["population"] = neigh["population"].fillna(0.0)
neigh["baseline_activity"] = neigh["baseline_activity"].fillna(0.0)

# -------------------------
# 6) Demand Score (adaptif ağırlık)
# -------------------------
# Eğer container verisi gerçekten geliyorsa container ağırlığını yükselt.
# Yoksa (çoğu 0 ise) nüfusu daha baskın yap.
has_container_signal = (neigh["container_total"] > 0).mean()  # 0'dan büyük oran
print(f"[FAZ3.1] container>0 oranı: {has_container_signal:.2%}")

if has_container_signal >= 0.30:
    # veri var: çok kaynaklı ağırlıklar
    w_container = 0.45
    w_population = 0.45
    w_baseline = 0.10
else:
    # container zayıf: nüfus ağırlıklı (sunumda açıkça belirtilir)
    w_container = 0.20
    w_population = 0.75
    w_baseline = 0.05

print(f"[FAZ3.1] ağırlıklar: container={w_container}, population={w_population}, baseline={w_baseline}")

neigh["z_container"] = zscore(neigh["container_total"])
neigh["z_population"] = zscore(neigh["population"])
neigh["z_baseline"] = zscore(neigh["baseline_activity"])

neigh["demand_raw"] = (
    w_container * neigh["z_container"] +
    w_population * neigh["z_population"] +
    w_baseline  * neigh["z_baseline"]
)

neigh["demand_score"] = minmax01(neigh["demand_raw"])

# -------------------------
# 7) Kaydet
# -------------------------
neigh_out = neigh[[
    "Mahalle_norm",
    "container_total", "population", "baseline_activity",
    "demand_raw", "demand_score"
]]
neigh_out.to_csv(OUT_COMP, index=False, encoding="utf-8-sig")

sp2 = sp.merge(neigh[["Mahalle_norm", "demand_score"]], on="Mahalle_norm", how="left")
sp2.to_csv(OUT_DEMAND, index=False, encoding="utf-8-sig")

daily = sp2.groupby(["date", "weekday_tr"], as_index=False).agg(
    neighborhoods=("Mahalle", "nunique"),
    total_demand=("demand_score", "sum"),
    avg_demand=("demand_score", "mean"),
    crane_neighborhoods=("crane_needed", "sum")
)
daily.to_csv(OUT_SUM, index=False, encoding="utf-8-sig")

print("\nFaz 3.1 tamam.")
print("Yazılan dosyalar:")
print(" -", OUT_COMP.resolve())
print(" -", OUT_DEMAND.resolve())
print(" -", OUT_SUM.resolve())

print("\nGünlük demand özeti (ilk 7):")
print(daily.head(7))

print("\nTop-5 mahalle (demand_score):")
print(neigh_out.sort_values("demand_score", ascending=False).head(5))

[FAZ3.1] container eşleşme oranı: 95.08%
[FAZ3.1] nüfus eşleşme oranı:     95.08%
[FAZ3.1] container>0 oranı: 95.08%
[FAZ3.1] ağırlıklar: container=0.45, population=0.45, baseline=0.1

Faz 3.1 tamam.
Yazılan dosyalar:
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\demand_components_neighborhood.csv
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\demand_by_date_neighborhood.csv
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\demand_summary_by_date.csv

Günlük demand özeti (ilk 7):
         date weekday_tr  neighborhoods  total_demand  avg_demand  \
0  2025-12-19       Cuma             38     11.486889    0.287172   
1  2025-12-20  Cumartesi             31     11.372933    0.355404   
2  2025-12-21      Pazar              2      1.539203    0.769602   
3  2025-12-22  Pazartesi             38     11.486889    0.287172   
4  2025-12-23       Salı             24      7.437825    0.297513   
5  2025-12-24   Çarşamba             38     11.486889    0.287172  

In [7]:
demand_summary_df = pd.read_csv("demand_summary_by_date.csv")
demand_summary_df.head()

,date,weekday_tr,neighborhoods,total_demand,avg_demand,crane_neighborhoods
0,2025-12-19,Cuma,38,11.486889,0.287172,5
1,2025-12-20,Cumartesi,31,11.372933,0.355404,8
2,2025-12-21,Pazar,2,1.539203,0.769602,0
3,2025-12-22,Pazartesi,38,11.486889,0.287172,5
4,2025-12-23,Salı,24,7.437825,0.297513,7


In [9]:
comp = pd.read_csv("demand_components_neighborhood.csv")
comp.sort_values("demand_score", ascending=False).head(30)


,Mahalle_norm,container_total,population,baseline_activity,demand_raw,demand_score
18,GORUKLE,841.000,32489.0,48910.285714,2.291225,1.000000
43,BALAT,990.000,21167.0,24818.571429,1.793329,0.842796
41,ATAEVLER,916.000,17793.0,14647.000000,1.458767,0.737162
44,BALKAN,826.000,19594.0,19075.142857,1.444851,0.732768
10,CALI,805.000,18226.0,27656.142857,1.384542,0.713726
34,UCEVLER,2.590,27559.0,144605.285714,1.286604,0.682804
38,23 NISAN,605.000,21042.0,15773.857143,1.184701,0.650629
56,KULTUR,862.000,13452.0,10858.428571,1.133605,0.634496
15,ESENTEPE,762.000,13096.0,21057.714286,1.020030,0.598636
22,IHSANIYE,1.296,28846.0,39592.142857,0.837516,0.541010


In [10]:
neigh[[
    "Mahalle_norm",
    "container_total",
    "population",
    "baseline_activity",
    "z_container",
    "z_population",
    "z_baseline",
    "demand_score"
]].sort_values("demand_score", ascending=False).head(10)


,Mahalle_norm,container_total,population,baseline_activity,z_container,z_population,z_baseline,demand_score
18,GORUKLE,841.000,32489.0,48910.285714,1.844253,2.855914,1.761504,1.000000
43,BALAT,990.000,21167.0,24818.571429,2.323893,1.532348,0.580201,0.842796
41,ATAEVLER,916.000,17793.0,14647.000000,2.085683,1.137921,0.081452,0.737162
44,BALKAN,826.000,19594.0,19075.142857,1.795967,1.348461,0.298580,0.732768
10,CALI,805.000,18226.0,27656.142857,1.728367,1.188539,0.719337,0.713726
34,UCEVLER,2.590,27559.0,144605.285714,-0.854639,2.279586,6.453771,0.682804
38,23 NISAN,605.000,21042.0,15773.857143,1.084555,1.517736,0.136706,0.650629
56,KULTUR,862.000,13452.0,10858.428571,1.911854,0.630449,-0.104315,0.634496
15,ESENTEPE,762.000,13096.0,21057.714286,1.589948,0.588832,0.395793,0.598636
22,IHSANIYE,1.296,28846.0,39592.142857,-0.858804,2.430039,1.304602,0.541010


In [11]:
neigh["demand_score"].describe()

count    61.000000
mean      0.276575
std       0.246906
min       0.000000
25%       0.078559
50%       0.195246
75%       0.429630
max       1.000000
Name: demand_score, dtype: float64

In [12]:
daily.sort_values("neighborhoods")[["date","neighborhoods","avg_demand"]]

,date,neighborhoods,avg_demand
2,2025-12-21,2,0.769602
4,2025-12-23,24,0.297513
1,2025-12-20,31,0.355404
6,2025-12-25,31,0.355404
0,2025-12-19,38,0.287172
3,2025-12-22,38,0.287172
5,2025-12-24,38,0.287172


In [13]:
sp2.groupby("crane_needed")["demand_score"].mean()

crane_needed
0    0.276753
1    0.481718
Name: demand_score, dtype: float64

In [14]:
sp2[sp2["demand_score"].isna()][["Mahalle"]].drop_duplicates()

,Mahalle
